# Analyzing Online Record Store Data Using SQL

## Open the db
Open the db file and look at basic information about which tables are available 

In [51]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [52]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Answer questions about the data

Write a query that returns each genre, with the number of tracks sold in the USA

In [53]:
%%sql
WITH
track_with_genre AS
(
    SELECT
        t.*,
        g.name genre
    FROM track t
    LEFT JOIN genre g ON g.genre_id = t.genre_id
),

track_with_num_sold AS
(
    SELECT
        t.*,
        il.invoice_id,
        il.quantity
    FROM track_with_genre t
    LEFT JOIN invoice_line il ON il.track_id = t.track_id
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id 
    WHERE c.country = 'USA'
)

SELECT 
    genre, 
    COUNT(*) count,
    100.0 * COUNT(*) / (SELECT COUNT(*) FROM track_with_num_sold) percentage
FROM track_with_num_sold
GROUP BY genre
ORDER BY count DESC;

 * sqlite:///chinook.db
Done.


genre,count,percentage
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.4253092293054235
Alternative,35,3.330161750713606
Pop,22,2.093244529019981
Latin,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

In [54]:
%%sql
WITH employee_with_sales AS
(
    SELECT
        e.first_name || ' ' || e.last_name name,
        SUM(i.total) total_sales,
        COUNT(DISTINCT c.customer_id) num_customers,
        SUM(i.total) / COUNT(DISTINCT c.customer_id) AS avg_sales_per_customer
    FROM employee e
    LEFT JOIN customer c ON c.support_rep_id = e.employee_id
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    WHERE e.title = 'Sales Support Agent'
    GROUP BY name
)

SELECT *
FROM employee_with_sales
ORDER BY total_sales DESC;

 * sqlite:///chinook.db
Done.


name,total_sales,num_customers,avg_sales_per_customer
Jane Peacock,1731.5100000000039,21,82.45285714285733
Margaret Park,1584.0000000000034,20,79.20000000000017
Steve Johnson,1393.920000000002,18,77.44000000000011
